In [ ]:
import pandas as pd
from pandas import DataFrame, Series

df = pd.read_csv('clusterData.csv')
df.head()

In [ ]:
df.isna().sum()

In [ ]:
pip install sklearn-pandas

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn_pandas import DataFrameMapper

# vectorizer = TfidfVectorizer(stop_words='english')
# features = vectorizer.fit_transform(documents)
#Add the features of the dataframe that you want to transform and/or combine
mapper = DataFrameMapper([
     ('label.value', TfidfVectorizer(stop_words='english')),
     ('abstract.value', TfidfVectorizer(stop_words='english')),
     ('words_string', TfidfVectorizer(stop_words='english'))
 ],df_out=True)

features = mapper.fit_transform(df)
X_cols = (
            mapper.features[0][1].get_feature_names()
            + mapper.features[1][1].get_feature_names()
            )
X_cols[50:60]

In [ ]:
mapper.features[0][1].get_feature_names()

In [ ]:
mapper = DataFrameMapper([
     ('label.value', TfidfVectorizer(stop_words='english')),
     ('abstract.value', TfidfVectorizer(stop_words='english')),
     ('words_string', TfidfVectorizer(stop_words='english'))
 ],df_out=True)

X = mapper.fit_transform(df)
X_cols = (
            mapper.features[0][1].get_feature_names()
            + mapper.features[1][1].get_feature_names()
            )



In [ ]:
#***********Extrass********
from sklearn.cluster import MiniBatchKMeans
# X = get_X(df)
kmeans = MiniBatchKMeans(n_clusters=50)
kmeans.fit(X)

In [ ]:
print("Top terms per cluster:")
order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
for i in range(50):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % X_cols[ind])

In [ ]:
print("Top terms per cluster:")
order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
print(order_centroids)
for i in range(50):
    print("Cluster %d:" % i)
    for ind in order_centroids[i, :10]:
        print(' %s' % X_cols[ind])

In [ ]:
features.head()

In [ ]:

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
sns.set()

from sklearn.cluster import KMeans

In [ ]:
wcss = []
# 'cl_num' is a that keeps track the highest number of clusters we want to use the WCSS method for.
# We have it set at 10 right now, but it is completely arbitrary.
cl_num = 135
for i in range(1,cl_num):
    
    kmeans = KMeans(i)
    
    kmeans.fit(features)
    
    wcss.append(kmeans.inertia_)
wcss

In [ ]:
# Plot the number of clusters vs WCSS
number_clusters = range(1,cl_num)
plt.plot(number_clusters,wcss)
plt.title('The Elbow Method')
plt.xlabel('Number of Clusters')
plt.ylabel('WCSS: Within-cluster Sum of Squares')

In [ ]:
# K-Means
from sklearn import cluster
import numpy as np
# Visualization and Analysis
import matplotlib.pyplot  as plt
import matplotlib.cm      as cm
import seaborn            as sns
from sklearn.metrics                  import silhouette_samples, silhouette_score
from wordcloud                        import WordCloud
import sklearn.metrics as metrics

In [ ]:
def run_KMeans(max_k, data):
    max_k += 1
    kmeans_results = dict()
    for k in range(2 , max_k):
        kmeans = cluster.KMeans(n_clusters = k
                               , init = 'k-means++'
                               , n_init = 10
                               , tol = 0.0001
                               , n_jobs = -1
                               , random_state = 1
                               , algorithm = 'full')

        kmeans_results.update( {k : kmeans.fit(features)} )
        
    return kmeans_results

In [ ]:
def printAvg(avg_dict):
    for avg in sorted(avg_dict.keys(), reverse=True):
        print("Avg: {}\tK:{}".format(avg.round(4), avg_dict[avg]))
        
def plotSilhouette(df, n_clusters, kmeans_labels, silhouette_avg):
    fig, ax1 = plt.subplots(1)
    fig.set_size_inches(8, 6)
    ax1.set_xlim([-0.2, 1])
    ax1.set_ylim([0, len(df) + (n_clusters + 1) * 10])
    
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--") # The vertical line for average silhouette score of all the values
    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.2, 0, 0.2, 0.4, 0.6, 0.8, 1])
    plt.title(("Silhouette analysis for K = %d" % n_clusters), fontsize=10, fontweight='bold')
    
    y_lower = 10
    sample_silhouette_values = silhouette_samples(df, kmeans_labels) # Compute the silhouette scores for each sample
    for i in range(n_clusters):
        ith_cluster_silhouette_values = sample_silhouette_values[kmeans_labels == i]
        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_silhouette_values, facecolor=color, edgecolor=color, alpha=0.7)

        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i)) # Label the silhouette plots with their cluster numbers at the middle
        y_lower = y_upper + 10  # Compute the new y_lower for next plot. 10 for the 0 samples
    plt.show()
    
        
def silhouette(kmeans_dict, df, plot=False):
    df = df.to_numpy()
    avg_dict = dict()
    for n_clusters, kmeans in kmeans_dict.items():      
        kmeans_labels = kmeans.predict(df)
        silhouette_avg = silhouette_score(df, kmeans_labels) # Average Score for all Samples
        avg_dict.update( {silhouette_avg : n_clusters} )
    
        if(plot): plotSilhouette(df, n_clusters, kmeans_labels, silhouette_avg)

In [ ]:
for i in range(3,13):
    labels=cluster.KMeans(n_clusters=i,init="k-means++",random_state=200).fit(features).labels_
    print ("Silhouette score for k(clusters) = "+str(i)+" is "
           +str(metrics.silhouette_score(features,labels,metric="euclidean",sample_size=1000,random_state=200)))

In [ ]:
k = 7
kmeans_results = run_KMeans(k, features)

In [ ]:
def get_top_features_cluster(tf_idf_array, prediction, n_feats):
    labels = np.unique(prediction)
    dfs = []
    for label in labels:
        id_temp = np.where(prediction==label) # indices for each cluster
        x_means = np.mean(tf_idf_array[id_temp], axis = 0) # returns average score across cluster
        sorted_means = np.argsort(x_means)[::-1][:n_feats] # indices with top 20 scores
        features = features.get_feature_names()
        best_features = [(features[i], x_means[i]) for i in sorted_means]
        df = pd.DataFrame(best_features, columns = ['features', 'score'])
        dfs.append(df)
    return dfs

def plotWords(dfs, n_feats):
    plt.figure(figsize=(8, 4))
    for i in range(0, len(dfs)):
        plt.title(("Most Common Words in Cluster {}".format(i)), fontsize=10, fontweight='bold')
        sns.barplot(x = 'score' , y = 'features', orient = 'h' , data = dfs[i][:n_feats])
        plt.show()

In [ ]:
best_result = 5
kmeans = kmeans_results.get(best_result)
final_df_array = features.to_numpy()
prediction = kmeans.predict(features)
n_feats = 20
dfs = get_top_features_cluster(final_df_array, prediction, n_feats)
plotWords(dfs, 13)

In [ ]:
type(features)

In [ ]:
df['cluster'] = kmeans.labels_
df.head(5)

In [ ]:
from sklearn.decomposition import PCA
random_state = 0 
pca = PCA(n_components=2, random_state=random_state)
reduced_features = pca.fit_transform(features)

# reduce the cluster centers to 2D
reduced_cluster_centers = pca.transform(kmeans.cluster_centers_)

In [ ]:
plt.scatter(reduced_features[:,0], reduced_features[:,1], c=kmeans.predict(features))
plt.scatter(reduced_cluster_centers[:, 0], reduced_cluster_centers[:,1], marker='x', s=150, c='b')